In [1]:
# for gioia's python build
import sys
sys.path.append('/Users/maggiori/anaconda/envs/py35/lib/python3.5/site-packages')

In [2]:
import numpy as np

In [3]:
# to import modules locally without having installed the entire package
# http://stackoverflow.com/questions/714063/importing-modules-from-parent-folder
import os, sys, inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0, parentdir)

In [4]:
from tsdb import PersistentDB
from tsdb import BitMapIndex
from timeseries import TimeSeries

In [5]:
# synthetic data
t = np.array([1, 1.5, 2, 2.5, 10, 11, 12])
v1 = np.array([10, 12, -11, 1.5, 10, 13, 17])
v2 = np.array([8, 12, -11, 1.5, 10, 13, 17])
a1 = TimeSeries(t, v1)
a2 = TimeSeries(t, v2)

In [6]:
identity = lambda x: x

# index: 1 is binary tree index, 2 is bitmap index
schema = {
  'pk':         {'type': 'str', 'convert': identity,   'index': None, 'values': None},
  'ts':         {'type': 'int', 'convert': identity,   'index': None, 'values': None},
  'order':      {'type': 'int', 'convert': int,        'index': 1, 'values': None},
  'blarg':      {'type': 'int', 'convert': int,        'index': 1, 'values': None},
  'useless':    {'type': 'int', 'convert': identity,   'index': 1, 'values': None},
  'mean':       {'type': 'float', 'convert': float,    'index': 1, 'values': None},
  'std':        {'type': 'float', 'convert': float,    'index': 1, 'values': None},
  'vp':         {'type': 'bool', 'convert': bool,      'index': 2, 'values': [True, False]},
  'deleted':    {'type': 'bool', 'convert': bool,      'index': 2, 'values': [True, False]}
}

In [7]:
# Delete any default db present (otherwise the db creation will load
# the previous one...)
filelist = ["db_files/" + f for f in os.listdir("db_files/")
            if f[:7] == 'default']
for f in filelist:
    os.remove(f)

In [8]:
# create persistent db
ddb = PersistentDB(schema, 'pk', len(t))

After init pks:  {}


In [9]:
ddb.insert_ts('pk1', a1)

Before insert, state of pks:  {}
After insert, state of pks:  {'pk1': (4, 0)}


In [10]:
ddb.upsert_meta('pk1', {'order': 1, 'blarg': 2})

In [11]:
ddb.insert_ts('pk2', a2)

Before insert, state of pks:  {'pk1': (4, 0)}
After insert, state of pks:  {'pk2': (116, 25), 'pk1': (4, 0)}


In [12]:
ddb.upsert_meta('pk2', {'order': 2, 'blarg': 2})

In [13]:
ddb.indexes

{'blarg': <tsdb.indexes.BinTreeIndex at 0x105e701d0>,
 'deleted': <tsdb.indexes.BitMapIndex at 0x105e70710>,
 'mean': <tsdb.indexes.BinTreeIndex at 0x105e70048>,
 'order': <tsdb.indexes.BinTreeIndex at 0x105e70ba8>,
 'std': <tsdb.indexes.BinTreeIndex at 0x105e70fd0>,
 'useless': <tsdb.indexes.BinTreeIndex at 0x105e70e80>,
 'vp': <tsdb.indexes.BitMapIndex at 0x105e70b00>}

In [14]:
# print metadata
for field, index in ddb.indexes.items():
    print(field, index.items())

std [(0.0, {'pk2', 'pk1'})]
deleted [(False, {'pk2', 'pk1'}), (True, set())]
mean [(0.0, {'pk2', 'pk1'})]
useless [(0, {'pk2', 'pk1'})]
vp [(False, {'pk2', 'pk1'}), (True, set())]
order [(0, set()), (1, {'pk1'}), (2, {'pk2'})]
blarg [(0, set()), (2, {'pk2', 'pk1'})]


In [15]:
ddb.pks.keys()

dict_keys(['pk2', 'pk1'])

In [16]:
ddb.pks.keys()

dict_keys(['pk2', 'pk1'])

In [17]:
ddb.select({'pk': 'pk1'}, [], None)

(['pk1'],
 [{'blarg': 2,
   'mean': 0.0,
   'order': 1,
   'pk': 'pk1',
   'std': 0.0,
   'useless': 0,
   'vp': False}])

In [18]:
ddb.select({'pk': 'pk2'}, [], None)

(['pk2'],
 [{'blarg': 2,
   'mean': 0.0,
   'order': 2,
   'pk': 'pk2',
   'std': 0.0,
   'useless': 0,
   'vp': False}])

In [19]:
ddb.delete_ts('pk1')

In [20]:
ddb.insert_ts('pk1', a1)

Before insert, state of pks:  {'pk2': (116, 25)}
After insert, state of pks:  {'pk2': (116, 25), 'pk1': (228, 50)}


In [21]:
# print metadata
for field, index in ddb.indexes.items():
    if isinstance(index, BitMapIndex):
        print(field)
        for k in index.keys():
            print(k, index[k])
        print('-----')

deleted
False {'pk2', 'pk1'}
True set()
-----
vp
False {'pk2', 'pk1'}
True set()
-----


In [22]:
ddb.insert_ts('pk3', a1)
ddb.insert_ts('pk4', a1)
ddb.insert_ts('pk5', a1)
ddb.insert_ts('pk6', a1)
ddb.insert_ts('pk7', a1)

Before insert, state of pks:  {'pk2': (116, 25), 'pk1': (228, 50)}
After insert, state of pks:  {'pk3': (340, 75), 'pk2': (116, 25), 'pk1': (228, 50)}
Before insert, state of pks:  {'pk3': (340, 75), 'pk2': (116, 25), 'pk1': (228, 50)}
After insert, state of pks:  {'pk3': (340, 75), 'pk2': (116, 25), 'pk4': (452, 100), 'pk1': (228, 50)}
Before insert, state of pks:  {'pk3': (340, 75), 'pk2': (116, 25), 'pk4': (452, 100), 'pk1': (228, 50)}
After insert, state of pks:  {'pk3': (340, 75), 'pk2': (116, 25), 'pk4': (452, 100), 'pk5': (564, 125), 'pk1': (228, 50)}
Before insert, state of pks:  {'pk3': (340, 75), 'pk2': (116, 25), 'pk4': (452, 100), 'pk5': (564, 125), 'pk1': (228, 50)}
After insert, state of pks:  {'pk3': (340, 75), 'pk6': (676, 150), 'pk1': (228, 50), 'pk2': (116, 25), 'pk4': (452, 100), 'pk5': (564, 125)}
Before insert, state of pks:  {'pk3': (340, 75), 'pk6': (676, 150), 'pk1': (228, 50), 'pk2': (116, 25), 'pk4': (452, 100), 'pk5': (564, 125)}
After insert, state of pks:  

In [23]:
# print metadata
for field, index in ddb.indexes.items():
    print(field)
    for k in index.keys():
        print(k, index[k])
    print('-----')

std
0.0 {'pk3', 'pk6', 'pk2', 'pk1', 'pk7', 'pk4', 'pk5'}
-----
deleted
False {'pk3', 'pk6', 'pk2', 'pk1', 'pk7', 'pk4', 'pk5'}
True set()
-----
mean
0.0 {'pk3', 'pk6', 'pk2', 'pk1', 'pk7', 'pk4', 'pk5'}
-----
useless
0 {'pk3', 'pk6', 'pk2', 'pk1', 'pk7', 'pk4', 'pk5'}
-----
vp
False {'pk3', 'pk6', 'pk2', 'pk1', 'pk7', 'pk4', 'pk5'}
True set()
-----
order
0 {'pk3', 'pk6', 'pk1', 'pk7', 'pk4', 'pk5'}
1 set()
2 {'pk2'}
-----
blarg
0 {'pk3', 'pk6', 'pk1', 'pk7', 'pk4', 'pk5'}
2 {'pk2'}
-----


In [24]:
ddb.upsert_meta('pk4', {'vp': True})
for field, index in ddb.indexes.items():
    if field == 'vp':
        for k in index.keys():
            print(k, index[k])

False {'pk3', 'pk6', 'pk2', 'pk1', 'pk7', 'pk5'}
True {'pk4'}


In [25]:
ddb.upsert_meta('pk5', {'vp': True})
for field, index in ddb.indexes.items():
    if field == 'vp':
        for k in index.keys():
            print(k, index[k])

False {'pk3', 'pk6', 'pk2', 'pk7', 'pk1'}
True {'pk4', 'pk5'}


In [26]:
ddb.upsert_meta('pk5', {'vp': False})
for field, index in ddb.indexes.items():
    if field == 'vp':
        for k in index.keys():
            print(k, index[k])

False {'pk3', 'pk6', 'pk2', 'pk1', 'pk7', 'pk5'}
True {'pk4'}


In [27]:
ddb.upsert_meta('pk5', {'vp': True})
for field, index in ddb.indexes.items():
    if field == 'vp':
        for k in index.keys():
            print(k, index[k])

False {'pk3', 'pk6', 'pk2', 'pk7', 'pk1'}
True {'pk4', 'pk5'}


In [28]:
ddb.delete_ts('pk5')

In [29]:
for field, index in ddb.indexes.items():
    if field == 'vp':
        for k in index.keys():
            print(k, index[k])

False {'pk3', 'pk6', 'pk2', 'pk7', 'pk1'}
True {'pk4'}


In [30]:
ddb.delete_ts('pk4')

In [31]:
for field, index in ddb.indexes.items():
    if field == 'vp':
        for k in index.keys():
            print(k, index[k])

False {'pk3', 'pk6', 'pk2', 'pk7', 'pk1'}
True set()


In [32]:
ddb.insert_ts('pk4', a1)
ddb.insert_ts('pk5', a1)
ddb.upsert_meta('pk4', {'vp': True})
ddb.upsert_meta('pk5', {'vp': True})

Before insert, state of pks:  {'pk3': (340, 75), 'pk6': (676, 150), 'pk1': (228, 50), 'pk2': (116, 25), 'pk7': (788, 175)}
After insert, state of pks:  {'pk3': (340, 75), 'pk6': (676, 150), 'pk1': (228, 50), 'pk2': (116, 25), 'pk4': (900, 200), 'pk7': (788, 175)}
Before insert, state of pks:  {'pk3': (340, 75), 'pk6': (676, 150), 'pk1': (228, 50), 'pk2': (116, 25), 'pk4': (900, 200), 'pk7': (788, 175)}
After insert, state of pks:  {'pk3': (340, 75), 'pk6': (676, 150), 'pk1': (228, 50), 'pk2': (116, 25), 'pk4': (900, 200), 'pk5': (1012, 225), 'pk7': (788, 175)}


In [33]:
ddb.select({'vp': True}, ['vp'], None)

(['pk4', 'pk5'], [{'vp': True}, {'vp': True}])

In [34]:
ddb.select({'vp': False}, ['vp'], None)

(['pk3', 'pk6', 'pk2', 'pk7', 'pk1'],
 [{'vp': False}, {'vp': False}, {'vp': False}, {'vp': False}, {'vp': False}])

In [35]:
# bitmap index errors??
print(ddb.indexes['vp'].values())
print(ddb.indexes['vp'].items())

[{'pk3', 'pk6', 'pk2', 'pk7', 'pk1'}, {'pk4', 'pk5'}]
[(False, {'pk3', 'pk6', 'pk2', 'pk7', 'pk1'}), (True, {'pk4', 'pk5'})]


In [36]:
[ddb.indexes['vp'][k] for k in ddb.indexes['vp'].keys()]

[{'pk1', 'pk2', 'pk3', 'pk6', 'pk7'}, {'pk4', 'pk5'}]

In [37]:
print(ddb.indexes['vp'][True])
print(ddb.indexes['vp'][False])

{'pk4', 'pk5'}
{'pk3', 'pk6', 'pk2', 'pk7', 'pk1'}


In [38]:
ddb = None